In [1]:
import h5py #loading data

import pandas as pd #work with data frame
import numpy as np #work with numerical
import math # Working with math problem

import matplotlib #for plotting data
import matplotlib.pyplot as plt #for plot data
import seaborn as sns #for pretty plot
import scipy.signal as signal #for general signal processing
import scipy as sci # for statistic

from biosppy.signals import ecg #for peak finding

import wfdb #for physionet tools
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
#import neurokit as nk #work with ECG data
#import peakutils #for basic peak finding utils
#import pywt #wavelet transform

In [2]:
# Plotting preferences
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [50, 10]  # Bigger figures
sns.set_style("whitegrid")  # White background
sns.set_palette(sns.color_palette("hls"))  # Better colours

In [3]:
dt = h5py.File('/Atrial Fibrillation/Data/hdf5_data.h5','r') #Load data
ref = pd.read_csv('/Atrial Fibrillation/Data/REFERENCE-v3.csv','r',header=None) # Load references
ls = list(dt.keys())
fs = 300 # Sampling Rate

In [4]:
# Calculate RR intervals in [s]
def RR_dist(Rpeaks):
    RR_dist = []
    for i in range(len(Rpeaks)-1):
        RR_dist.append((Rpeaks[i+1] - Rpeaks[i])/fs)
        #RR_dist.append((Rpeaks[i+1] - Rpeaks[i]))
    return np.array(RR_dist)
# Calculate adjacent RR intervals
def RR_adj_dif(RR_dist):
    RR_adj_dif = []
    for i in range(len(RR_dist)-1):
        RR_adj_dif.append(RR_dist[i+1] - RR_dist[i])
    return np.array(RR_adj_dif)
# Count those RR intervals greater than 50ms
def NN50(RR_dist): 
    NN50 = 0
    for i in range(len(RR_dist)-1):
        if abs(RR_dist[i+1] - RR_dist[i]) > 0.05: NN50 +=1
    return NN50
# Normalized Absolute Deviation
def nadev(rr_dist, MRR):
    alpha = 0.75; beta = 1.4
    RRi = rr_dist[(rr_dist > MRR*alpha) & (rr_dist < MRR*beta)]
    nadev = np.sum(abs((RRi-MRR))/(len(RRi)*MRR))
    return nadev
# Normalized Absolute Difference 
def nadiff(rr_adj_dif, MRR):
    gamma = 0.3; 
    RRi_dif = rr_adj_dif[abs(rr_adj_dif) <= gamma]
    nadiff = np.sum(abs(RRi_dif)/(MRR*len(RRi_dif)))
    return nadiff
#Finding the statistic feature of P wave 
#Finding the statistic feature of P wave 
def P_stats(Rpeaks, p_min_dist, p_max_dist):
    p1_sum = 0; p2_sum = 0; p3_sum = 0; p4_sum = 0; p5_sum = 0; p6_sum = 0 # Sum of each segments of P wave
    tmp_Rpeaks = Rpeaks[4:]
    for i, peak in enumerate(tmp_Rpeaks,0):
        tmp = de_dt[(peak+p_min_dist) : (peak+p_max_dist+1)]
        tmp0 = tmp - np.mean(tmp) #Demean 
        count = math.ceil(len(tmp0)/6) # Divide into 6 segments
        tmp1 = tmp0[0:count] 
        tmp2 = tmp0[count:count*2]
        tmp3 = tmp0[count*2:count*3]
        tmp4 = tmp0[count*3:count*4]
        tmp5 = tmp0[count*4:count*5]
        tmp6 = tmp0[count*5:] # 6 segments
        p1_sum += np.mean(tmp1); p2_sum += np.mean(tmp2); p3_sum += np.mean(tmp3); p4_sum += np.mean(tmp4); p5_sum += np.mean(tmp5); p6_sum += np.mean(tmp6) # Summation of mean of P waves'segments      
        f = np.zeros((len(Rpeaks),3))
        f[i][0] = np.var(tmp0) # Variance of the P wave at ith R peak
        f[i][1] = sci.stats.skew(tmp0) # Skewness of the P at ith wave R peak
        f[i][2] = sci.stats.kurtosis(tmp0) # Kurtosis of the P wave at ith R peak
        return (p1_sum/len(tmp_Rpeaks), p2_sum/len(tmp_Rpeaks), p3_sum/len(tmp_Rpeaks), 
                p4_sum/len(tmp_Rpeaks), p5_sum/len(tmp_Rpeaks), p6_sum/len(tmp_Rpeaks), 
                f.mean(0)[0],f.mean(0)[1],f.mean(0)[2])
#Finding the statistic feature of RR segments => each RR is divided into segments. 
def rrseg_stats(Rpeaks):
    tmp_Rpeaks = Rpeaks[4:]
    for i in range(len(tmp_Rpeaks)):
        tmp = de_dt[tmp_Rpeaks[i] : tmp_Rpeaks[i+1]] # RR interval
        tmp0 = tmp - np.mean(tmp) #Demean 
        count = math.ceil(len(tmp0)/6) # Divide into 6 segments

         # 6 segments
        tmp1 = tmp0[0:count] 
        tmp2 = tmp0[count:count*2]
        tmp3 = tmp0[count*2:count*3]
        tmp4 = tmp0[count*3:count*4]
        tmp5 = tmp0[count*4:count*5]
        tmp6 = tmp0[count*5:] # 6 segments
        
        # Feature of 1st RR segment
        rr1 = np.zeros((len(tmp_Rpeaks)-1, 4)) # Feature of 1st RR segment
        rr1[i][0] = np.mean(tmp1) # Mean of 1st RR segment at ith RR interval
        rr1[i][1] = np.var(tmp1) # Variance of 1st RR segment at ith RR interval
        rr1[i][2] = sci.stats.skew(tmp1) # Skewness of 1st RR segment at ith RR interval
        rr1[i][3] = sci.stats.kurtosis(tmp1) # Kurtosis of 1st RR segment at ith RR interval
        
        # Feature of 2sd RR segment
        rr2 = np.zeros((len(tmp_Rpeaks)-1, 4)) # Feature of 1st RR segment
        rr2[i][0] = np.mean(tmp2) # Mean of 1st RR segment at ith RR interval
        rr2[i][1] = np.var(tmp2) # Variance of 1st RR segment at ith RR interval
        rr2[i][2] = sci.stats.skew(tmp2) # Skewness of 1st RR segment at ith RR interval
        rr2[i][3] = sci.stats.kurtosis(tmp2) # Kurtosis of 1st RR segment at ith RR interval
        
        # Feature of 3rd RR segment
        rr3 = np.zeros((len(tmp_Rpeaks)-1, 4)) # Feature of 1st RR segment
        rr3[i][0] = np.mean(tmp3) # Mean of 1st RR segment at ith RR interval
        rr3[i][1] = np.var(tmp3) # Variance of 1st RR segment at ith RR interval
        rr3[i][2] = sci.stats.skew(tmp3) # Skewness of 1st RR segment at ith RR interval
        rr3[i][3] = sci.stats.kurtosis(tmp3) # Kurtosis of 1st RR segment at ith RR interval
        
        # Feature of 4th RR segment
        rr4 = np.zeros((len(tmp_Rpeaks)-1, 4)) # Feature of 1st RR segment
        rr4[i][0] = np.mean(tmp4) # Mean of 1st RR segment at ith RR interval
        rr4[i][1] = np.var(tmp4) # Variance of 1st RR segment at ith RR interval
        rr4[i][2] = sci.stats.skew(tmp4) # Skewness of 1st RR segment at ith RR interval
        rr4[i][3] = sci.stats.kurtosis(tmp4) # Kurtosis of 1st RR segment at ith RR interval
        
        # Feature of 5th RR segment
        rr5 = np.zeros((len(tmp_Rpeaks)-1, 4)) # Feature of 1st RR segment
        rr5[i][0] = np.mean(tmp5) # Mean of 1st RR segment at ith RR interval
        rr5[i][1] = np.var(tmp5) # Variance of 1st RR segment at ith RR interval
        rr5[i][2] = sci.stats.skew(tmp5) # Skewness of 1st RR segment at ith RR interval
        rr5[i][3] = sci.stats.kurtosis(tmp5) # Kurtosis of 1st RR segment at ith RR interval
        
        # Feature of 6th RR segment
        rr6 = np.zeros((len(tmp_Rpeaks)-1, 4)) # Feature of 1st RR segment
        rr6[i][0] = np.mean(tmp6) # Mean of 1st RR segment at ith RR interval
        rr6[i][1] = np.var(tmp6) # Variance of 1st RR segment at ith RR interval
        rr6[i][2] = sci.stats.skew(tmp6) # Skewness of 1st RR segment at ith RR interval
        rr6[i][3] = sci.stats.kurtosis(tmp6) # Kurtosis of 1st RR segment at ith RR interval
        
        return(rr1.mean(0)[0], rr1.mean(0)[1], rr1.mean(0)[2], rr1.mean(0)[3],
               rr2.mean(0)[0], rr2.mean(0)[1], rr2.mean(0)[2], rr2.mean(0)[3],
               rr3.mean(0)[0], rr3.mean(0)[1], rr3.mean(0)[2], rr3.mean(0)[3],
               rr4.mean(0)[0], rr4.mean(0)[1], rr4.mean(0)[2], rr4.mean(0)[3],
               rr5.mean(0)[0], rr5.mean(0)[1], rr5.mean(0)[2], rr5.mean(0)[3],
               rr6.mean(0)[0], rr6.mean(0)[1], rr6.mean(0)[2], rr6.mean(0)[3])

In [5]:
# Create a list of features
f = pd.DataFrame(index=ls,columns=['rr_dist','rr_adj_dif','MRR','SDNN','RMSSD','pNN50','CV','delta_RRI_max','min_RR',
                                   'median_RR','mean_R','std_R','NADev','NADiff','p1','p2','p3','p4','p5','p6','Pvar',
                                   'Pskew','Pkurt','rr1_mean','rr1_var','rr1_skew','rr1_kurt','rr2_mean','rr2_var','rr2_skew',
                                   'rr2_kurt','rr3_mean','rr3_var','rr3_skew','rr3_kurt','rr4_mean','rr4_var','rr4_skew','rr4_kurt',
                                   'rr5_mean','rr5_var','rr5_skew','rr5_kurt','rr6_mean','rr6_var','rr6_skew','rr6_kurt','Avg_Pow','Label'])

In [ ]:
try:
    for i in range(len(ls)):
        tmp = np.ndarray.flatten(np.transpose(np.array(dt.get(ls[i]))))
        low_pass = 45 # Cut-off frequency
        high_pass = 0.2 # Cut-off frequency
        a, b = signal.butter(6, (high_pass, low_pass), btype='bandpass', analog=True)
        de_dt = signal.lfilter(b, a, tmp) #Denoised Data
        rpeaks, = ecg.hamilton_segmenter(signal=de_dt, sampling_rate=fs) # Find R peaks 
        Rpeaks, = ecg.correct_rpeaks(signal=de_dt, rpeaks=rpeaks, sampling_rate=fs, tol=0.05) # correct R-peak locations
        # RR features 
        f['rr_dist'][i] = RR_dist(Rpeaks) #RR intervals
        f['rr_adj_dif'][i] = RR_adj_dif(f['rr_dist'][i]) # Adjacent RR intervals
        f['MRR'][i] = np.mean(f['rr_dist'][i]) # Mean of all RR intervals
        f['SDNN'][i] = np.std(f['rr_dist'][i]) #Std of all RR intervals
        f['RMSSD'][i] = np.sqrt(np.sum(np.square(f['rr_adj_dif'][i]))/(len(Rpeaks)-2))
        f['pNN50'][i] = NN50(f['rr_dist'][i])/(len(f['rr_dist'][i])*100)
        f['CV'][i] = f['SDNN'][i]/f['MRR'][i] # Coefficient of Variance of RR intervals
        f['delta_RRI_max'][i] = max(f['rr_dist'][i]) - min(f['rr_dist'][i]) 
        f['min_RR'][i] = min(f['rr_dist'][i]) # Minimum of all RR intervals
        f['median_RR'][i] = np.median(f['rr_dist'][i]) # Median of all RR intervals
        # R features
        f['mean_R'][i] = np.mean(de_dt[Rpeaks]) # Average of all R-peaks amplitude
        f['std_R'][i] = np.std(de_dt[Rpeaks]) # Standard deviation of all R-peaks amplitude
        # Normalized 
        f['NADev'][i] = nadev(f['rr_dist'][i], f['MRR'][i])
        f['NADiff'][i] = nadiff(f['rr_adj_dif'][i], f['MRR'][i])
        #Finding the statistic feature of P wave 
        p_min_dist = -60
        p_max_dist = -20
        [f['p1'][i], f['p2'][i], f['p3'][i], f['p4'][i], f['p5'][i], f['p6'][i], f['Pvar'][i], f['Pskew'][i], f['Pkurt'][i]] = np.array(P_stats(Rpeaks, p_min_dist, p_max_dist)) # p1, p2, p3, p4, p5, p6, Pvar, Pskew, Pkurt
        # RR interval' segments features
        [f['rr1_mean'][i], f['rr1_var'][i], f['rr1_skew'][i], f['rr1_kurt'][i], 
         f['rr2_mean'][i], f['rr2_var'][i], f['rr2_skew'][i], f['rr2_kurt'][i],
         f['rr3_mean'][i], f['rr3_var'][i], f['rr3_skew'][i], f['rr3_kurt'][i],
         f['rr4_mean'][i], f['rr4_var'][i], f['rr4_skew'][i], f['rr4_kurt'][i],
         f['rr5_mean'][i], f['rr5_var'][i], f['rr5_skew'][i], f['rr5_kurt'][i],
         f['rr6_mean'][i], f['rr6_var'][i], f['rr6_skew'][i], f['rr6_kurt'][i]]= np.array(rrseg_stats(Rpeaks))
        # Average power
        f['Avg_Pow'][i] = np.sum(de_dt**2)/(len(de_dt)/fs)
        # Label of the ecg
        for rf in ref[0]: 
            if (rf.split(',')[0] == f.index[i].split('.')[0]):
                f['Label'][i] = rf.split(',')[1]
except ZeroDivisionError:
    print('There is an error at:',f.index[i])

c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\numpy\core\fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\numpy\core\

In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

encoder_index = LabelEncoder()
f['target_label'] = encoder_index.fit_transform(f['Label'])
X = f.drop(['rr_dist','rr_adj_dif','Label','target_label','rr5_mean','rr5_var','rr5_skew','rr5_kurt','rr6_mean','rr6_var','rr6_skew','rr6_kurt'], axis='columns').astype(float)
# Z = X = f.drop(['rr_dist','rr_adj_dif','Label','target_label','rr1_mean','rr1_var','rr1_skew','rr1_kurt','rr2_mean','rr2_var','rr2_skew','rr2_kurt','rr3_mean','rr3_var','rr3_skew','rr3_kurt','rr4_mean','rr4_var','rr4_skew','rr4_kurt','rr5_mean','rr5_var','rr5_skew','rr5_kurt','rr6_mean','rr6_var','rr6_skew','rr6_kurt'], axis='columns').astype(float)
y = f.target_label
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
# Z_train, Z_test, y_train, y_test = train_test_split(Z,y, test_size=0.2)
# Scale Features
min_max_scaler = MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_test_minmax = min_max_scaler.fit_transform(X_test)

In [8]:
#importing library and segregation of data as train and test using DMatrix Data structure
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [9]:
model = xgb.XGBClassifier(silent=False,
                          min_child_weight=20,
                          learning_rate=0.2,
                          colsample_bytree = 0.9,
                          subsample = 0.8,
                          objective='multi:softmax',
                          n_estimators=100,
                          max_depth=11,
                          gamma=1,
                          num_class=4)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))

[16:22:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 10 pruned nodes, max_depth=8
[16:22:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 10 pruned nodes, max_depth=10
[16:22:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 10 pruned nodes, max_depth=10
[16:22:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 8 pruned nodes, max_depth=5
[16:22:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 10 pruned nodes, max_depth=8
[16:22:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 12 pruned nodes, max_depth=11
[16:22:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree 

In [ ]:
# Export the pipeline as a python script file for further utilize
model.save_model('AF_classify_d11.py')

In [ ]:
cv = np.mean(cross_val_score(model, X_train, y_train, cv=8))
print ("Accuracy using RF with 10 cross validation : {}%".format(round(cv*100,2)))

In [ ]:
f.head()

In [ ]:
model = 